model

In [1]:
!pip install transformers==3.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 769 kB 22.6 MB/s 
     |████████████████████████████████| 3.0 MB 40.4 MB/s 
     |████████████████████████████████| 1.3 MB 35.3 MB/s 
     |████████████████████████████████| 880 kB 40.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=36a1a68e7899a248bcec0256f398bc172b7e75d667e259045744e7a37cf75a35
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [2]:
def mount_drive():  
    from google.colab import drive
    drive.mount('/content/gdrive')

In [3]:
if __name__ == "__main__":
    mount_drive()

Mounted at /content/gdrive


In [4]:
import torch
from transformers import get_linear_schedule_with_warmup, BertForSequenceClassification, BertConfig

In [5]:
from torch import nn
from transformers import BertModel, BertConfig

class BertClassifier(nn.Module):

    # BertClassifier는 초대용량 AI 모델인 BertModel을 파인튜닝한 모델이다.
    # 구체적으로, BertModel 위에 linear한 층 하나를 추가해 놓은 모델이다.
    # Bert -> Linear -> Softmax의 순서로 연산이 이루어진다.
    # 우리가 Loss를 구하는데 사용되는 CrossEntropyLoss의 경우, softmax를 자동으로 포함하여 연산해준다.
    # 고로, BertClassifier 클래스 내의 forward 함수에, softmax는 포함시키지 않는다.
    # 단, 학습의 용도가 아니라 테스트나 실제 시연을 할 때는 forward 결과물에 softmax를 적용시켜줘야 한다.

    def __init__(self, num_labels=7):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('monologg/kobert')
        self.linear = nn.Linear(768, num_labels)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids= input_ids, attention_mask=attention_mask)
        linear_output = self.linear(pooled_output)

        return linear_output

In [6]:
# 분류를 위한 BERT 모델 생성
# BertModel을 초기화하는 역할
def BertModelInitialization():
    PATH = "model.pt"

    # BertModel은 다양한 작업을 진행할 수 있도록 해주는 여러 인터페이스들을 제공한다.
    # 그 중, 본 중간 미션의 task에 가장 적합한 인터페이스를 찾아보자.
    # Hint 1 : HuggingFace 홈페이지에 Bert를 검색해서 찾아보자.
    # URL : https://huggingface.co/docs/transformers/main/en/index
    # Hint 2 : 본 중간 미션이, 영화리뷰를 긍정과 부정의 두 가지 감정으로 분류해내는 작업임을 고려해보자.

    model = BertClassifier()

    # 생성한 모델을 특정 PATH에 저장
    torch.save(model.state_dict(), PATH)

In [7]:
def get_model(device, cuda_available):
    PATH = "model.pt"

    model = BertClassifier()

    if cuda_available:
        # PATH에 저장된 모델을 불러오기
        model.load_state_dict(torch.load(PATH), strict=False)

        # 불러온 모델을 device에 올리기
        model = model.to(device)
    else:
        # PATH에 저장된 모델을 불러오기 및 불러온 모델을 device에 올리기
        model.load_state_dict(torch.load(PATH, map_location=device), strict=False)
    
    return model

신경망 성능 향상을 위한 다양한 툴

- 옵티마이저, 에포크 수, 스케줄러.
- 오버피팅 방지: early stopping, drop out, l1 정규화 및 l2 정규화 등.




In [8]:
def get_model_with_params(num_data, device, cuda_available):
    model = get_model(device, cuda_available)

    # 옵티마이저 설정
    optimizer = torch.optim.AdamW(model.parameters(),
                      lr = 1e-5, # 학습률
                      eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값,
                    )
    # 전체 데이터가 총 몇 번 학습되는지
    epochs = 3

    # 총 훈련 스텝
    total_steps = num_data * epochs

    # 학습이 이루어짐에 따라 learning_rate을 감소시키기 위한 스케줄러
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0,
                                                num_training_steps = total_steps)
    #손실함숫값 계산
    criterion = nn.CrossEntropyLoss()
    
    return model, optimizer, scheduler, epochs, criterion


In [9]:
def main():
    %cd /content/gdrive/MyDrive/NLP
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    BertModelInitialization()
    print(get_model_with_params(34388, device, torch.cuda.is_available()))

In [10]:
if __name__ == '__main__':
      main()

/content/gdrive/MyDrive/NLP


Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

(BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru